In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
import torchvision.transforms as transforms 
import torch
from torch.utils.data import DataLoader

In [4]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(
    root = './data',
    train = True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root = './data',
    train = False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bottom = nn.Sequential( # -> 1 x 28 x 28
			nn.Conv2d(1 , 16, 3, stride=1), # -> 16 x 26 x 26 
			nn.ReLU(),
            nn.AvgPool2d(2, stride=1), # -> 16 x 25 x 25
            nn.Conv2d(16 , 33, 3, stride=1), # -> 33 x 23 x 23
			nn.ReLU(),
            nn.AvgPool2d(3, stride=1), # -> 33 x 21 x 21
		)
        
        self.classifier = nn.Sequential(
			nn.Linear(33 * 21 * 21, 20),
            nn.Linear(20, 10)
		)
    
    def forward(self, x):
        x = self.bottom(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
lenet = LeNet()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(2):
    running_loss = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = lenet(inputs)

        if isinstance(labels, int):
            labels = torch.tensor([labels])

        labels = labels.long()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


In [8]:
correct = 0
total = 0

lenet.eval()

with torch.no_grad():
    for images, labels in test_loader:
        outputs = lenet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test data: {100 * correct / total:.2f}%')


Accuracy on test data: 90.35%


In [10]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # должно быть, например, '11.8'
print(torch.cuda.device_count())


2.7.1+cpu
None
0
